In [1]:
import os 
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from IPython.display import Markdown

In [3]:
GOOGLE_API_KEY=os.environ['GOOGLE_API_KEY']

In [4]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_google_genai import GoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain.chains import RetrievalQA

/Users/aayushchaudhary/Git/college/literature-bot/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Load the Paper

`Set proper name of the path to the paper's pdf`

If loading single dcoument: 

In [7]:
loader = PyPDFLoader('./papers/0.pdf')
data = loader.load()

Multiple Docs

`here set the directory where you have the papers`

In [8]:
loader = DirectoryLoader(path='./papers/', glob='**/*.pdf', loader_cls=PyPDFLoader)
data = loader.load()

Data is loaded page by page

In [28]:
data[12].page_content

'S2275 Attention-based graph neural networks: a\xa0survey  \n1 3\nInter-layer GATs: This kind of works usually select features beyond neural network \nlayers with multiple feature spaces, not just local neighborhoods. Across the neural net-\nwork layer, attention in inter-layer GATs can be regarded as an operation of cross-layer \nfusion of different feature spaces with feature fusion attention. In this term, attention-\nbased GNNs dynamically select features from different levels, different channels, dif-\nferent views, or different time slices. Therefore, we further divide these methods into \nfive sub-categories (i.e., multi-level attention (Liu et\xa0 al. 2020; Zhang et\xa0 al. 2022c), \nmulti-channel (Bo et\xa0al. 2021; Luan et\xa0al. 2021), multi-view (Wang et\xa0al. 2020b; Yuan \net\xa0al. 2021b), Spatio-temporal attention (Sankar et\xa0al. 2018; Lu et\xa0al. 2019), and time \nseries attention (Zhang et\xa0al. 2021c; Zhao et\xa0al. 2020)). By considering temporal attrib-\nutes, 

### Split all the docs into Chunks

Split pages into 1000 word chunks with buffer/overlap of 200 words

In [9]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size= 1000, chunk_overlap=200)
texts = text_splitter.split_documents(data)

In [ ]:
texts[0].page_content

Generate model embeddings

In [10]:
embeddings = GoogleGenerativeAIEmbeddings(model='models/embedding-001', google_api_key=GOOGLE_API_KEY)
vector_store = Chroma.from_documents(documents=texts, embedding=embeddings)

I0000 00:00:1722946400.395771   43860 check_gcp_environment_no_op.cc:29] ALTS: Platforms other than Linux and Windows are not supported


Setup Retriver

In [11]:
retriever = vector_store.as_retriever()

Setup LLM to be used and chain

In [22]:
llm = GoogleGenerativeAI(model="gemini-1.5-flash", google_api_key=GOOGLE_API_KEY)

In [13]:
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff", 
    retriever=retriever,
    return_source_documents=True
)

A Query to generate review of all papers: 

`Should contain all the names of your papers`

In [29]:
query = ("""
You are a [Research Assitant] bot. You help with creating [Literature Review]. 

Input :You will be given access to a [research paper]


Task: You have to [extract] the following information:
Information To BE extracted is present in backticks: 

Paper Name
Focus Area of the paper
Date: month/year
Methodology: (eg Qualitative, Quantitative, Review, Conceptual, Report)
Key Findings: in 10 words what the paper has implemented/achieved
Application: real life potential use cases (summerize in 10 words or 1-2 points)
Challenges:  Drawbacks of this paper/approach (summerize 1-2 short points)
Opportunities: Future scope/possibilities of paper (summerize in 1-2 points)
Dataset: (dataset used in the paper if available else write none) 

Note: if including multiple points, insert newline character or write a unordered list in markdown wrapped as a string

Output Format is inside triple backticks: 
```
python
[
        
{
"paper_name": "name of the paper 0",
.
.
.
"opportunities": "..."
"datatset": "name of the dataset"
},
{
"paper_name": "name of the paper 1",
.
.
.
"opportunities": "..."
"datatset": "name of the dataset"
}
...
]

```
         
Names of the papers: 
<<<
A Survey on Graph Neural Networks for Time
Series: Forecasting, Classification, Imputation,
and Anomaly Detection, 
Attention Is All You Need,
Exploring Hierarchical Structures for Recommender Systems,
Heterogeneous Graph Attention Network, 
Hyperbolic Graph Attention Network, 
KGAT: Knowledge Graph Attention Network for Recommendation, 
Knowledge Graph Embedding Based on Graph Neural Network,
Research on the application of Nerual Network model in knowledge graph completion technology,
Attention based graph neural networks: a survey, 
Graph neural networks for visual question answering: a systematic review, 
A review of graph neural networks: concepts, architectures, techniques, challenges, datasets, applications, and future directions
>>>
""")

### Run the chain

In [30]:
result = qa_chain.invoke({'query': query})

Result is Stored in `result` object and can be accessed using key: 

In [31]:
result['result']

'```python\n[\n{\n"paper_name": "A Survey on Graph Neural Networks for Time Series: Forecasting, Classification, Imputation, and Anomaly Detection",\n"Focus Area of the paper": "Graph Neural Networks for Time Series",\n"Date": "12/2022",\n"Methodology": "Review",\n"Key Findings": "Surveyed GNNs for time series tasks",\n"Application": "Forecasting, classification, imputation, anomaly detection",\n"Challenges": "Limited data availability, model complexity",\n"Opportunities": "Explore new architectures, improve interpretability",\n"datatset": "None"\n},\n{\n"paper_name": "Attention Is All You Need",\n"Focus Area of the paper": "Attention Mechanism in Neural Networks",\n"Date": "06/2017",\n"Methodology": "Quantitative",\n"Key Findings": "Proposed Transformer architecture based on attention",\n"Application": "Machine translation, text summarization",\n"Challenges": "High computational cost, lack of interpretability",\n"Opportunities": "Explore new attention mechanisms, improve efficiency",\

### Our Output is formatted in a way that it can be parsed as a python list of dicts/JSON objects

In [32]:
Markdown(result['result'])

```python
[
{
"paper_name": "A Survey on Graph Neural Networks for Time Series: Forecasting, Classification, Imputation, and Anomaly Detection",
"Focus Area of the paper": "Graph Neural Networks for Time Series",
"Date": "12/2022",
"Methodology": "Review",
"Key Findings": "Surveyed GNNs for time series tasks",
"Application": "Forecasting, classification, imputation, anomaly detection",
"Challenges": "Limited data availability, model complexity",
"Opportunities": "Explore new architectures, improve interpretability",
"datatset": "None"
},
{
"paper_name": "Attention Is All You Need",
"Focus Area of the paper": "Attention Mechanism in Neural Networks",
"Date": "06/2017",
"Methodology": "Quantitative",
"Key Findings": "Proposed Transformer architecture based on attention",
"Application": "Machine translation, text summarization",
"Challenges": "High computational cost, lack of interpretability",
"Opportunities": "Explore new attention mechanisms, improve efficiency",
"datatset": "WMT 2014 English-to-German translation task"
},
{
"paper_name": "Exploring Hierarchical Structures for Recommender Systems",
"Focus Area of the paper": "Hierarchical Structures in Recommender Systems",
"Date": "01/2019",
"Methodology": "Quantitative",
"Key Findings": "Proposed hierarchical attention network for recommendation",
"Application": "Recommending products, services, content",
"Challenges": "Data sparsity, cold-start problem",
"Opportunities": "Explore new hierarchical structures, improve personalization",
"datatset": "MovieLens 100K, Amazon Reviews"
},
{
"paper_name": "Heterogeneous Graph Attention Network",
"Focus Area of the paper": "Heterogeneous Graph Attention Networks",
"Date": "08/2019",
"Methodology": "Quantitative",
"Key Findings": "Proposed HGAN for heterogeneous graph learning",
"Application": "Node classification, link prediction",
"Challenges": "Handling different node types and relations",
"Opportunities": "Explore new attention mechanisms for heterogeneous graphs",
"datatset": "Amazon co-purchasing network, DBLP citation network"
},
{
"paper_name": "Hyperbolic Graph Attention Network",
"Focus Area of the paper": "Hyperbolic Graph Attention Networks",
"Date": "02/2020",
"Methodology": "Quantitative",
"Key Findings": "Proposed HGAT for learning on hyperbolic graphs",
"Application": "Node classification, link prediction",
"Challenges": "Handling non-Euclidean space, model complexity",
"Opportunities": "Explore new hyperbolic embedding techniques",
"datatset": "Cora, PubMed, Amazon"
},
{
"paper_name": "KGAT: Knowledge Graph Attention Network for Recommendation",
"Focus Area of the paper": "Knowledge Graph Attention Networks for Recommendation",
"Date": "06/2019",
"Methodology": "Quantitative",
"Key Findings": "Proposed KGAT for incorporating knowledge graphs into recommendation",
"Application": "Recommending products, services, content",
"Challenges": "Scalability, knowledge graph incompleteness",
"Opportunities": "Explore new ways to integrate knowledge graphs",
"datatset": "Amazon, MovieLens"
},
{
"paper_name": "Knowledge Graph Embedding Based on Graph Neural Network",
"Focus Area of the paper": "Knowledge Graph Embedding using GNNs",
"Date": "03/2020",
"Methodology": "Quantitative",
"Key Findings": "Proposed GNN-based embedding for knowledge graphs",
"Application": "Knowledge graph completion, question answering",
"Challenges": "Data sparsity, model complexity",
"Opportunities": "Explore new GNN architectures for knowledge graphs",
"datatset": "FB15k-237, WN18RR"
},
{
"paper_name": "Research on the application of Nerual Network model in knowledge graph completion technology",
"Focus Area of the paper": "Knowledge Graph Completion using Neural Networks",
"Date": "06/2020",
"Methodology": "Review",
"Key Findings": "Surveyed NN models for knowledge graph completion",
"Application": "Knowledge graph completion, question answering",
"Challenges": "Data sparsity, model complexity",
"Opportunities": "Explore new NN architectures for knowledge graphs",
"datatset": "None"
},
{
"paper_name": "Attention based graph neural networks: a survey",
"Focus Area of the paper": "Attention Mechanisms in Graph Neural Networks",
"Date": "06/2021",
"Methodology": "Review",
"Key Findings": "Surveyed attention-based GNNs",
"Application": "Node classification, link prediction, graph classification",
"Challenges": "Computational cost, interpretability",
"Opportunities": "Explore new attention mechanisms, improve efficiency",
"datatset": "None"
},
{
"paper_name": "Graph neural networks for visual question answering: a systematic review",
"Focus Area of the paper": "Graph Neural Networks for Visual Question Answering",
"Date": "08/2021",
"Methodology": "Review",
"Key Findings": "Surveyed GNNs for visual question answering",
"Application": "Visual question answering, image captioning",
"Challenges": "Data scarcity, model complexity",
"Opportunities": "Explore new GNN architectures for visual tasks",
"datatset": "None"
},
{
"paper_name": "A review of graph neural networks: concepts, architectures, techniques, challenges, datasets, applications, and future directions",
"Focus Area of the paper": "Graph Neural Networks",
"Date": "12/2020",
"Methodology": "Review",
"Key Findings": "Comprehensive review of GNNs",
"Application": "Node classification, link prediction, graph classification",
"Challenges": "Scalability, interpretability, data sparsity",
"Opportunities": "Explore new GNN architectures, improve interpretability",
"datatset": "None"
}
]
```

# Final Automated Generation

A cleaner approach to run one by one: 
- will help with limited context
- possibly better accuracy and generation

In [5]:
system_promp_template = ("""
You are a [Research Assitant] bot. You help with creating [Literature Review]. 

Input :You will be given access to a [research paper]


Task: You have to [extract] the following information:
Information To BE extracted is present in backticks: 

Paper Name
Focus Area of the paper
Date: month/year
Methodology: (eg Qualitative, Quantitative, Review, Conceptual, Report)
Key Findings: in 10 words what the paper has implemented/achieved
Application: real life potential use cases (summerize in 10 words or 1-2 points)
Challenges:  Drawbacks of this paper/approach (summerize 1-2 short points)
Opportunities: Future scope/possibilities of paper (summerize in 1-2 points)
Dataset: (dataset used in the paper if available else write none) 

Note: if including multiple points, insert newline character or write a unordered list in markdown wrapped as a string

Output Format is inside triple backticks: 
```
json      
{
"paper_name": "name of the paper 0",
.
.
.
"opportunities": "..."
"datatset": "name of the dataset"
}
```
              
""")

In [6]:
user_query_template = """
{system_prompt_template}
Generate a literature review based on this research paper
{paper_name}
"""

In [7]:
from langchain.prompts import PromptTemplate

final_prompt_template= PromptTemplate.from_template(user_query_template)

**All our processing for the paper consolidated into one function** 

In [13]:
def process_paper(file_path, paper_name: str): 
    loader = PyPDFLoader(file_path)
    data = loader.load()

    text_splitter = RecursiveCharacterTextSplitter(chunk_size= 1000, chunk_overlap=200)
    texts = text_splitter.split_documents(data)
    
    embeddings = GoogleGenerativeAIEmbeddings(model='models/embedding-001', google_api_key=GOOGLE_API_KEY)
    vector_store = Chroma.from_documents(documents=texts, embedding=embeddings)

    retriever = vector_store.as_retriever()
    
    llm = GoogleGenerativeAI(model="gemini-1.5-flash", google_api_key=GOOGLE_API_KEY)

    qa_chain = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff", 
        retriever=retriever,
        return_source_documents=True    
    )


    final_prompt = final_prompt_template.format(
        system_prompt_template=system_promp_template, 
        paper_name =paper_name
    )
    result = qa_chain.invoke({'query': final_prompt})
    
    return result
    

- A simple functions to parseout the JSON
- Alternatively you can setup JSONOutputParser from langchain.outputparsers (reqires additional setup)

In [9]:
import json
def extract_and_append_json(json_string):
    # Remove the ```json and ``` markers if present
    json_string = json_string.strip()
    if json_string.startswith("```json"):
        json_string = json_string[7:]
    if json_string.endswith("```"):
        json_string = json_string[:-3]
    
    # Parse the JSON string
    try:
        json_data = json.loads(json_string)
        
        # Append the parsed JSON to the result list
        return json_data

    except json.JSONDecodeError as e:
        print(e)

Name of all the papers you're working with

In [34]:

paper_names = [
"Self-Supervised Learning of Graph Neural Networks",
"""A Survey on Graph Neural Networks for Time
Series: Forecasting, Classification, Imputation,
and Anomaly Detection""", 
"""Attention Is All You Need""",
"""Exploring Hierarchical Structures for Recommender Systems""",
"""Heterogeneous Graph Attention Network""", 
"""Hyperbolic Graph Attention Network""", 
"""KGAT: Knowledge Graph Attention Network for Recommendation""", 
"""Knowledge Graph Embedding Based on Graph Neural Network""",
"""Research on the application of Nerual Network model in knowledge graph completion technology""",
"""Attention based graph neural networks: a survey""", 
"""Graph neural networks for visual question answering: a systematic review""", 
"""A review of graph neural networks: concepts, architectures, techniques, challenges, datasets, applications, and future directions"""
]


Call Chain on each paper individually and parse output into dicts, stored into `reslist`

In [42]:
papers_directory = './papers'

#stores the result
reslist = []
ct = 0

for filename in os.listdir(papers_directory):
    if ct > 9: break
    if ct == 2: pass
    if filename.endswith(".pdf"):
        file_path = os.path.join(papers_directory, filename)
        print(f"Processing: {file_path}")
        num = int(filename[:-4])


        result = process_paper(file_path, paper_names[ct])

        print(f"Processed Paper: {paper_names[ct]}")

        reslist.append(extract_and_append_json(result['result']))
    ct+=1

reslist

Processing: ./papers/9.pdf


I0000 00:00:1723138646.874906  475288 check_gcp_environment_no_op.cc:29] ALTS: Platforms other than Linux and Windows are not supported
I0000 00:00:1723138656.991248  475288 check_gcp_environment_no_op.cc:29] ALTS: Platforms other than Linux and Windows are not supported


Processed Paper: Self-Supervised Learning of Graph Neural Networks
Processing: ./papers/8.pdf


I0000 00:00:1723138660.854935  475288 check_gcp_environment_no_op.cc:29] ALTS: Platforms other than Linux and Windows are not supported
I0000 00:00:1723138665.050557  475288 check_gcp_environment_no_op.cc:29] ALTS: Platforms other than Linux and Windows are not supported


Processed Paper: A Survey on Graph Neural Networks for Time
Series: Forecasting, Classification, Imputation,
and Anomaly Detection
Processing: ./papers/10.pdf


...

I0000 00:00:1723138734.203401  475288 check_gcp_environment_no_op.cc:29] ALTS: Platforms other than Linux and Windows are not supported
I0000 00:00:1723138743.754238  475288 check_gcp_environment_no_op.cc:29] ALTS: Platforms other than Linux and Windows are not supported


Processed Paper: Research on the application of Nerual Network model in knowledge graph completion technology
Processing: ./papers/1.pdf


I0000 00:00:1723138748.337893  475288 check_gcp_environment_no_op.cc:29] ALTS: Platforms other than Linux and Windows are not supported
I0000 00:00:1723138760.845032  475288 check_gcp_environment_no_op.cc:29] ALTS: Platforms other than Linux and Windows are not supported


Processed Paper: Attention based graph neural networks: a survey


[{'paper_name': 'Self-Supervised Learning of Graph Neural Networks',
  'focus_area_of_the_paper': 'Graph Neural Networks (GNNs)',
  'date': '01/2024',
  'methodology': 'Review',
  'key_findings': 'Review of self-supervised learning for GNNs',
  'application': 'Graph classification, node classification, link prediction',
  'challenges': 'Lack of large-scale labeled graph data, difficulty in designing effective self-supervised tasks',
  'opportunities': 'Development of new self-supervised learning methods for GNNs, exploration of applications in various domains',
  'dataset': 'None'},
 {'paper_name': 'A Survey on Graph Neural Networks for Time Series: Forecasting, Classification, Imputation, and Anomaly Detection',
  'focus_area': 'Graph Neural Networks for Time Series Analysis',
  'date': 'August/2021',
  'methodology': 'Review',
  'key_findings': 'Taxonomy of GNNs for time series tasks',
  'application': 'Forecasting, Classification, Imputation, Anomaly Detection',
  'challenges': 'Lim

```Accessing One Object```

In [57]:
reslist[8]

{'paper_name': 'Knowledge Graph Completion Based on Recurrent Neural Network',
 'focus_area': 'Knowledge Graph Completion',
 'date': '01/2023',
 'methodology': 'Quantitative',
 'key_findings': 'Recurrent neural network for knowledge graph completion',
 'application': 'Predicting missing links in knowledge graphs',
 'challenges': '-  Limited scalability to large knowledge graphs\n-  Difficulty in handling complex relationships',
 'opportunities': '-  Improving performance with advanced neural network architectures\n-  Exploring applications in various domains',
 'dataset': 'None'}

In [56]:
Markdown(reslist[1]['opportunities'])

Develop novel GNN architectures for specific time series tasks, explore applications in diverse domains

### We can also create another chain that takes in these json lists and turn those into MD

Why not then just return MD instead of JSON in first place? 
- allows to process each paper individually
- can feed into other chains easily
- cleaner and structured approach


In [58]:
create_markdown_tempalte = """
Given a list of JSON objects inside triple backticks. \
Generate a table in [Markdown] using the keys as Table Columns

```{list_of_papers}```
"""

md_prompt = PromptTemplate.from_template(create_markdown_tempalte)

fin_prompt = md_prompt.format(
    list_of_papers=reslist
)

In [59]:
llm = llm 

In [60]:
lit_review_md = llm.invoke(fin_prompt)

### Visualize: 

In [61]:
Markdown(lit_review_md)

| paper_name | focus_area_of_the_paper | date | methodology | key_findings | application | challenges | opportunities | dataset |
|---|---|---|---|---|---|---|---|---|
| Self-Supervised Learning of Graph Neural Networks | Graph Neural Networks (GNNs) | 01/2024 | Review | Review of self-supervised learning for GNNs | Graph classification, node classification, link prediction | Lack of large-scale labeled graph data, difficulty in designing effective self-supervised tasks | Development of new self-supervised learning methods for GNNs, exploration of applications in various domains | None |
| A Survey on Graph Neural Networks for Time Series: Forecasting, Classification, Imputation, and Anomaly Detection | Graph Neural Networks for Time Series Analysis | August/2021 | Review | Taxonomy of GNNs for time series tasks | Forecasting, Classification, Imputation, Anomaly Detection | Limited research on multi-step forecasting, need for standardized benchmarks | Develop novel GNN architectures for specific time series tasks, explore applications in diverse domains | None |
| Attention Is All You Need | Natural Language Processing (NLP), Machine Translation | 6/2017 | Quantitative, Conceptual | Proposes Transformer architecture for NLP tasks | Machine translation, text summarization, question answering | Computational complexity, lack of interpretability | Further research on Transformer variants, applications in other domains | WMT 2014 English-to-German and English-to-French translation tasks |
| Heterogeneous Graph Attention Network | Graph Neural Networks | None | Quantitative | Proposes a heterogeneous graph attention network | Knowledge graph completion, recommendation systems | - Difficulty in handling heterogeneous graphs<br>- Scalability issues | - Explore new applications of heterogeneous graph networks<br>- Improve scalability and efficiency | None |
| Hyperbolic Graph Attention Network | Graph Neural Networks | 01/2022 | Quantitative | Developed a novel hyperbolic graph attention network | -  Node classification in complex networks<br>-  Recommendation systems | -  Computational complexity of hyperbolic geometry<br>-  Hyperbolic embedding of large graphs | -  Applications in other graph-related tasks<br>-  Exploration of different hyperbolic geometries | None |
| KGAT: Knowledge Graph Attention Network for Recommendation | Recommender Systems, Knowledge Graph Embedding | August/2019 | Quantitative | Attention-based knowledge graph embedding for recommendation | Personalized recommendation, Explainable AI | Computational complexity, Scalability for large knowledge graphs | Integration with other recommendation techniques, Exploration of different attention mechanisms | Amazon-book, Last-FM, Yelp2018 |
| Knowledge Graph Embedding Based on Graph Neural Network | Knowledge Graph Embedding | N/A | Review | Survey of GNNs for knowledge graph embedding | Improve knowledge graph completion and reasoning | Scalability of GNNs for large graphs<br>Data sparsity in knowledge graphs | Explore new GNN architectures<br>Develop efficient training methods | None |
| Knowledge Graph Completion Based on Recurrent Neural Network | Knowledge Graph Completion | 01/2023 | Quantitative | Recurrent neural network for knowledge graph completion | Predicting missing links in knowledge graphs | -  Limited scalability to large knowledge graphs<br>-  Difficulty in handling complex relationships | -  Improving performance with advanced neural network architectures<br>-  Exploring applications in various domains | None |
| Attention-based graph neural networks: a survey | Attention-based graph neural networks | 01/2023 | Review | Classifies attention-based GNNs into three stages | Various applications in machine learning, such as natural language processing, computer vision, and recommender systems | -  Limited theoretical understanding of attention-based GNNs<br>-  Computational complexity of attention mechanisms | -  Development of more efficient and effective attention mechanisms<br>-  Exploration of new applications for attention-based GNNs | None | 


Done

In [62]:
print(lit_review_md)

| paper_name | focus_area_of_the_paper | date | methodology | key_findings | application | challenges | opportunities | dataset |
|---|---|---|---|---|---|---|---|---|
| Self-Supervised Learning of Graph Neural Networks | Graph Neural Networks (GNNs) | 01/2024 | Review | Review of self-supervised learning for GNNs | Graph classification, node classification, link prediction | Lack of large-scale labeled graph data, difficulty in designing effective self-supervised tasks | Development of new self-supervised learning methods for GNNs, exploration of applications in various domains | None |
| A Survey on Graph Neural Networks for Time Series: Forecasting, Classification, Imputation, and Anomaly Detection | Graph Neural Networks for Time Series Analysis | August/2021 | Review | Taxonomy of GNNs for time series tasks | Forecasting, Classification, Imputation, Anomaly Detection | Limited research on multi-step forecasting, need for standardized benchmarks | Develop novel GNN architectures fo

# IDEA GENERATION - generate ideas for Paper

In [70]:
suggest_ideas_tempalte = """
You are a `Machine Learning` professor and we your students need to [work on final year project]. 
For that we have to come up with a list of papers to refer to. 

`Based on them we have to come up with some research gaps we can fulfil in certain domain or topic` with the goal of: 
`Write a Research Paper`
and `Create the final project`. 

We need your help to come up with <potential areas> our team can address as final year B.E. students. 

The project has to build upon existing research and its helpful if we create something that is `trending` in research like LLMs and VLM etc. 

Our areas of Interest: 
- Knowledge Representation (very important)
- Graph Neural Networks 
- GAT
- Deep Learning 
- ML 
- LLMs
- NLP
- Multimodal AI

`our idea has to be novel`


END GOAL: 
<
- Come up with 3-5`objectives` that we can have as outcomes of our project
- final projects that we can do (END TO END Ideas)
- workflow or project plan for that project 
- scope of project should be 3 months
- expose us to a wide range of experiences
- help come up with names for the project 
>


Given a list of papers as `Literature Review` in Markdown format.

```{lit_review}```


Output Format: 
```markdown
# Project Title: Title of Project
# Paper Title: Title of Paper
# Abstract: 5-10 sentences
# Motivation: Why do we want to build on this research (4 bullet points)
# Objectives: <IDEAS that we are introducing newly to the field> (3-4 bullet points) <- should contain `keywords`
# Architecture: (you can't draw just tell the steps or names of them)
# Application Areas: `3-5` areas where our research can have an impact
# conclusion: 
# Project Plan: 
```


constraints: 
```
- It should not be too hard to develop for beginners 
- Should have a small scope not requiring years of scope
- At the end explain what all topics we need to study within 3 weeks to be knoledge-ible enough to make this project. 
```


COME UP WITH ONLY ONE IDEA
"""

md_prompt = PromptTemplate.from_template(suggest_ideas_tempalte)

synopsis_format = md_prompt.format(
    lit_review=lit_review_md
)

In [64]:
llm = GoogleGenerativeAI(model="gemini-1.5-flash", google_api_key=GOOGLE_API_KEY, temperature=0.7)

In [71]:
synopsis = llm.invoke(synopsis_format)

In [72]:
Markdown(synopsis)

## Project Title:  Multimodal Knowledge Graph Completion with Self-Supervised Learning for Enhanced Reasoning

## Paper Title: Self-Supervised Learning of Graph Neural Networks

## Abstract: 

This project aims to leverage self-supervised learning techniques to enhance knowledge graph completion, specifically focusing on multimodal knowledge graphs that integrate textual and visual information. By extending existing self-supervised learning methods for graph neural networks (GNNs) to handle multimodal data, we propose a novel approach to improve the accuracy and reasoning capabilities of knowledge graph completion systems. 

## Motivation:

* **Limited Labelled Data:** Knowledge graph completion tasks often struggle with limited labelled data, hindering the performance of traditional supervised learning methods.
* **Multimodal Information:**  Real-world knowledge graphs are increasingly multimodal, incorporating textual descriptions, images, and videos. Leveraging this rich information can significantly enhance knowledge representation and reasoning.
* **Self-Supervised Learning:** Self-supervised learning offers a promising approach to address the data scarcity problem by learning meaningful representations from unlabeled data.
* **GNNs for Knowledge Graphs:** GNNs have proven effective in capturing complex relationships within knowledge graphs, but their performance can be further improved by incorporating multimodal information and leveraging self-supervised learning.

## Objectives:

* **Multimodal Knowledge Graph Embedding:** Develop a novel GNN architecture for embedding multimodal knowledge graphs, effectively integrating textual and visual information.
* **Self-Supervised Learning for Multimodal GNNs:** Design and implement self-supervised learning tasks specifically tailored for multimodal GNNs to learn robust representations from unlabeled data.
* **Enhanced Reasoning Capabilities:** Evaluate the proposed approach on various knowledge graph completion tasks, demonstrating improved accuracy and reasoning capabilities compared to existing methods.
* **Explainable Knowledge Graph Completion:** Explore methods to provide interpretable explanations for the predicted knowledge graph completions, enhancing the transparency and trust in the system.

## Architecture:

The proposed architecture will consist of:

* **Multimodal Encoder:**  A GNN-based encoder that processes both textual and visual features from the knowledge graph entities.
* **Self-Supervised Learning Module:** A module that generates self-supervised learning tasks based on the multimodal features, such as masked entity prediction or multimodal contrastive learning.
* **Knowledge Graph Completion Module:**  A module that leverages the learned multimodal embeddings to predict missing links in the knowledge graph.

## Application Areas:

* **Recommendation Systems:** Enhance recommendation systems by leveraging multimodal knowledge about user preferences and product information.
* **Question Answering:**  Improve question answering systems by integrating knowledge from multimodal knowledge graphs, allowing for more comprehensive and informative responses.
* **Drug Discovery:** Facilitate drug discovery by identifying potential drug candidates through knowledge graph completion and reasoning over multimodal data about drug-target interactions.
* **Social Media Analysis:** Analyze social media data by constructing multimodal knowledge graphs and leveraging them to understand user behavior, sentiment, and trends.

## Conclusion: 

This project aims to significantly advance the field of knowledge graph completion by integrating multimodal information and leveraging self-supervised learning techniques. The proposed approach holds the potential to improve the accuracy and reasoning capabilities of knowledge graph completion systems, enabling impactful applications across various domains.

## Project Plan:

**Phase 1 (Weeks 1-2):**

* **Literature Review:**  Thorough review of existing research on knowledge graph completion, multimodal knowledge graphs, self-supervised learning for GNNs, and explainable AI.
* **Data Collection:** Identify and collect suitable multimodal knowledge graph datasets for the project.
* **Model Selection:** Choose a suitable GNN architecture for multimodal knowledge graph embedding.
* **Self-Supervised Task Design:** Develop specific self-supervised learning tasks for the chosen GNN architecture.

**Phase 2 (Weeks 3-4):**

* **Model Implementation:** Implement the proposed GNN architecture and self-supervised learning module.
* **Training and Evaluation:** Train the model on the chosen dataset and evaluate its performance on various knowledge graph completion tasks.
* **Analysis and Visualization:** Analyze the results and visualize the learned multimodal embeddings to gain insights into the model's performance.

**Phase 3 (Weeks 5-6):**

* **Explainability Exploration:** Explore techniques to provide interpretable explanations for the predicted knowledge graph completions.
* **Application Development:** Develop a proof-of-concept application demonstrating the potential impact of the proposed approach in a specific domain (e.g., recommendation systems, question answering).
* **Report Writing:** Prepare a detailed project report documenting the research process, findings, and contributions.

**Phase 4 (Week 7):**

* **Presentation and Defense:** Present the project findings and defend the research to a panel of experts.

## Topics to Study:

* **Knowledge Representation and Reasoning:** Learn about different knowledge representation schemes, knowledge graph structures, and reasoning techniques.
* **Graph Neural Networks:** Gain a thorough understanding of GNN architectures, their applications, and how they learn from graph data.
* **Self-Supervised Learning:**  Study different self-supervised learning approaches and their applications in machine learning.
* **Multimodal Data Processing:**  Learn about techniques for handling and processing multimodal data, including textual and visual information.
* **Explainable AI:** Understand the importance of explainability in machine learning models and explore different methods for achieving it.

This project will provide a comprehensive understanding of knowledge graph completion, self-supervised learning, and multimodal data processing. It will also equip students with the skills to develop innovative solutions for real-world problems. 


In [73]:
Markdown(llm.invoke(synopsis_format))

## Project Title:  Knowledge-Enhanced Multimodal Graph Attention Network for Personalized Recommendation

## Paper Title:  KGAT: Knowledge Graph Attention Network for Recommendation

## Abstract: 

This project aims to enhance personalized recommendation systems by leveraging the power of knowledge graphs and multimodal data. We propose a novel Knowledge-Enhanced Multimodal Graph Attention Network (KEM-GAT) that integrates knowledge graph embeddings with visual and textual information to provide more accurate and explainable recommendations. KEM-GAT utilizes a multi-head attention mechanism to capture complex relationships between users, items, and knowledge entities, leading to improved recommendation performance.

## Motivation:

* **Limitations of Traditional Recommender Systems:** Existing recommendation systems primarily rely on user-item interactions, neglecting valuable contextual information from knowledge graphs and multimodal data.
* **Knowledge Graph Integration:** Incorporating knowledge graphs can enhance recommendation accuracy by providing rich semantic relationships between entities, leading to better understanding of user preferences and item attributes.
* **Multimodal Data Fusion:** Combining visual and textual information can provide a more comprehensive representation of items, leading to more relevant and engaging recommendations.
* **Explainable Recommendations:** Our approach aims to provide explainable recommendations by highlighting the knowledge entities and multimodal features that influence the recommendation process, fostering user trust and understanding.

## Objectives:

* **Develop a novel KEM-GAT architecture:** This architecture will integrate knowledge graph embeddings, visual features (e.g., images), and textual features (e.g., descriptions) into a unified graph representation.
* **Implement a multi-head attention mechanism:** This mechanism will allow the model to capture complex relationships between users, items, and knowledge entities, leading to more accurate and personalized recommendations.
* **Evaluate the performance of KEM-GAT on real-world recommendation datasets:** We will compare KEM-GAT's performance against existing recommendation methods, demonstrating its effectiveness in capturing user preferences and providing relevant recommendations.
* **Develop a visualization tool to analyze the influence of knowledge entities and multimodal features on recommendations:** This tool will provide insights into the decision-making process of the model, enhancing its explainability and transparency.

## Architecture:

1. **Knowledge Graph Embedding:** Embed knowledge entities from a knowledge graph into a low-dimensional vector space.
2. **Multimodal Feature Extraction:** Extract visual features from images and textual features from item descriptions using pre-trained models.
3. **Multimodal Graph Construction:** Combine user, item, knowledge entity, and multimodal feature nodes into a heterogeneous graph, connecting them based on their relationships.
4. **KEM-GAT Layer:** Apply a multi-head attention mechanism to learn complex relationships between nodes within the multimodal graph.
5. **Recommendation Generation:** Predict user preferences and generate personalized recommendations based on the learned representations.

## Application Areas:

* **E-commerce:** Recommending products based on user preferences, purchase history, and product information from a knowledge graph.
* **Social Media:** Recommending content based on user interests, social connections, and topic-related knowledge entities.
* **News Recommendation:** Recommending news articles based on user profiles, reading history, and current events from a knowledge graph.
* **Movie Recommendation:** Recommending movies based on user ratings, genre preferences, and movie information from a knowledge graph.

## Conclusion:

The KEM-GAT model presents a promising approach to enhance personalized recommendation systems by leveraging the power of knowledge graphs and multimodal data. By integrating knowledge entities and multimodal features into a unified graph representation, KEM-GAT captures complex relationships between users, items, and knowledge entities, leading to more accurate, explainable, and personalized recommendations.

## Project Plan:

**Month 1:**

* **Literature Review:** Study relevant papers on knowledge graph embedding, multimodal data fusion, and graph neural networks.
* **Dataset Selection:** Choose a suitable recommendation dataset that includes user-item interactions, knowledge graph information, and multimodal features.
* **Knowledge Graph Construction:** Construct a knowledge graph based on the chosen dataset, including relevant entities and relationships.

**Month 2:**

* **Multimodal Feature Extraction:** Implement pre-trained models to extract visual and textual features from the dataset.
* **KEM-GAT Architecture Implementation:** Develop the KEM-GAT architecture, including the multi-head attention mechanism.
* **Model Training and Evaluation:** Train and evaluate the KEM-GAT model on the chosen dataset, comparing its performance to existing recommendation methods.

**Month 3:**

* **Visualization Tool Development:** Create a visualization tool to analyze the influence of knowledge entities and multimodal features on recommendations.
* **Project Report Writing:** Document the project findings, including the methodology, results, and discussion.
* **Project Presentation:** Prepare and present the project results to an audience.

## Topics to Study:

* **Knowledge Graph Embedding:** Techniques for representing entities and relationships from a knowledge graph in a low-dimensional vector space.
* **Multimodal Data Fusion:** Methods for combining visual, textual, and other modalities of data for improved representation and understanding.
* **Graph Neural Networks:** Architectures and algorithms for learning from graph-structured data.
* **Attention Mechanisms:** Techniques for selectively focusing on relevant information within a dataset.
* **Recommendation Systems:** Principles and techniques for personalized recommendation systems.

## Project Names:

* Knowledge-Enhanced Multimodal Recommendations
* Graph-Powered Personalized Recommendations
* Multimodal Graph Attention for Recommendation
* Explainable Recommendations with Knowledge Graphs
* Personalized Recommendations with Multimodal Insights 


In [74]:
Markdown(llm.invoke(synopsis_format))

## Project Title:  Hyperbolic Knowledge Graph Embedding for Enhanced Recommendation Systems

## Paper Title: Hyperbolic Graph Attention Network (HGAT)

## Abstract: 

This project proposes an enhanced knowledge graph embedding (KGE) model for recommendation systems based on the Hyperbolic Graph Attention Network (HGAT). HGAT leverages the inherent hierarchical structure of knowledge graphs by embedding entities and relationships in hyperbolic space, allowing for more efficient representation of complex relationships and improved recommendation accuracy. We aim to address the limitations of existing KGE methods in handling large, complex knowledge graphs and explore the potential of HGAT for personalized and explainable recommendations.

## Motivation:

* **Scalability:** Existing KGE methods struggle to scale to large, complex knowledge graphs, limiting their effectiveness in real-world applications.
* **Explanatory Power:** Many recommendation systems lack transparency, making it difficult to understand the reasoning behind recommendations.
* **Hierarchical Representation:** Knowledge graphs exhibit inherent hierarchical structures, which are not effectively captured by traditional Euclidean embedding spaces.
* **Improved Recommendation Accuracy:** Hyperbolic space provides a more natural representation for hierarchical data, potentially leading to better performance in recommendation tasks.

## Objectives:

* **Develop a novel HGAT-based KGE model:**  We aim to design an optimized HGAT architecture specifically tailored for KGE and recommendation tasks.
* **Enhance recommendation accuracy:** We will evaluate the performance of our HGAT-based model on standard recommendation datasets and compare it with existing KGE methods.
* **Improve explainability of recommendations:** We will explore techniques to interpret the predictions made by our model, providing insights into the reasoning behind recommendations.
* **Investigate the impact of different hyperbolic geometries:** We will experiment with different hyperbolic geometries and analyze their effect on model performance and explainability.

## Architecture:

* **Knowledge Graph Construction:** Build a knowledge graph from a chosen recommendation dataset.
* **Hyperbolic Embedding:** Embed entities and relationships into a hyperbolic space using HGAT.
* **Recommendation Generation:** Utilize the embedded knowledge graph to generate recommendations based on user preferences and item characteristics.
* **Explainability Module:** Develop a mechanism to interpret the recommendations and provide insights into the reasoning process.

## Application Areas:

* **E-commerce:** Personalized product recommendations based on user history and item relationships.
* **Social Media:**  Personalized content recommendations based on user interests and social connections.
* **Movie/Music Recommendation:**  Suggesting movies or music based on user preferences and genre relationships.
* **Healthcare:** Recommending treatments or therapies based on patient history and medical knowledge.

## Conclusion:

This project aims to contribute to the development of more efficient and effective KGE models for recommendation systems. By leveraging the power of HGAT and exploring its potential for explainability, we seek to improve the accuracy and transparency of personalized recommendations.

## Project Plan:

**Phase 1 (Week 1-2):**
* Literature Review:  Deep dive into KGE, HGAT, and recommendation systems.
* Dataset Selection: Choose a relevant recommendation dataset and construct a knowledge graph.
* Model Architecture: Design the HGAT-based KGE model and develop the necessary components.

**Phase 2 (Week 3-4):**
* Implementation: Implement the HGAT model using a deep learning framework (e.g., PyTorch).
* Training and Evaluation: Train the model on the chosen dataset and evaluate its performance against baseline methods.

**Phase 3 (Week 5-6):**
* Explainability Module: Design and implement a module to interpret the recommendations and provide insights.
* Experimentation: Investigate different hyperbolic geometries and analyze their impact on performance.
* Report Writing: Document the project findings, methodology, and contributions.

**Phase 4 (Week 7-8):**
* Presentation Preparation: Prepare a detailed presentation of the project and its outcomes.
* Final Presentation: Present the project to a panel of experts and peers.

## Topics to Study:

* **Knowledge Representation:**  Understand different knowledge representation techniques, including RDF, OWL, and knowledge graphs.
* **Graph Neural Networks:** Learn about various GNN architectures, including GAT, GCN, and their applications.
* **Hyperbolic Geometry:**  Explore the basics of hyperbolic geometry and its applications in machine learning.
* **Recommendation Systems:**  Study different recommendation algorithms, including collaborative filtering, content-based filtering, and knowledge-based recommendation.
* **Explainable AI:**  Learn about techniques for interpreting and explaining the predictions of machine learning models. 
* **Deep Learning Frameworks:**  Gain proficiency in using a deep learning framework like PyTorch or TensorFlow.

This project provides a challenging and rewarding opportunity to explore cutting-edge research in knowledge graph embedding and recommendation systems. By focusing on the novel application of HGAT and addressing real-world challenges, this project has the potential to make a significant impact in the field of personalized AI. 


In [75]:
Markdown(llm.invoke(synopsis_format))

## Project Title:  KG-LLM: Enhancing Knowledge Graph Reasoning with Large Language Models

## Paper Title:  Knowledge Graph Completion Based on Recurrent Neural Network

## Abstract: 

This project aims to leverage the power of Large Language Models (LLMs) to enhance knowledge graph reasoning, specifically focusing on the task of knowledge graph completion. We propose a novel framework that integrates LLMs with existing knowledge graph embedding techniques, enabling more accurate and robust prediction of missing relationships in knowledge graphs. Our approach utilizes the vast knowledge and reasoning abilities of LLMs to generate contextualized representations of entities and relations, which are then used to refine the embeddings learned by traditional knowledge graph embedding models. This fusion of LLM capabilities with graph neural networks addresses the limitations of current methods, particularly their struggle with handling complex relationships and scalability to large knowledge graphs.

## Motivation:

* **Addressing limitations of current knowledge graph completion methods:** Existing methods often struggle with capturing complex relationships and scaling to large knowledge graphs. 
* **Leveraging the power of LLMs for improved reasoning:** LLMs possess vast knowledge and reasoning abilities, offering a powerful tool for enhancing knowledge graph completion.
* **Bridging the gap between symbolic and neural approaches:**  Our framework combines the strengths of both symbolic knowledge representation (knowledge graphs) and neural reasoning (LLMs) for a more comprehensive approach.
* **Enabling more accurate and robust knowledge graph completion:** By incorporating LLM knowledge and reasoning, our approach aims to significantly improve the accuracy and robustness of knowledge graph completion.

## Objectives:

* **Develop a novel hybrid framework for knowledge graph completion:**  This framework will integrate LLMs with existing knowledge graph embedding techniques.
* **Investigate the effectiveness of using LLMs for generating contextualized entity and relation representations:** We will explore various techniques for utilizing LLMs to enrich knowledge graph embeddings.
* **Evaluate the performance of our proposed framework on benchmark knowledge graph datasets:** We will compare our approach against state-of-the-art knowledge graph completion methods.
* **Explore applications of our enhanced knowledge graph in downstream tasks:**  We will investigate the potential of our framework for improving knowledge-based tasks like question answering and recommendation systems.

## Architecture:

**Step 1:**  Pre-train an LLM on a large corpus of text and code.
**Step 2:**  Utilize the pre-trained LLM to generate contextualized representations of entities and relations within the target knowledge graph.
**Step 3:**  Embed these contextualized representations into a vector space using a suitable knowledge graph embedding technique (e.g., TransE, RotatE).
**Step 4:**  Train a knowledge graph completion model using the enriched embeddings, enabling it to predict missing relationships in the knowledge graph.

## Application Areas:

* **Knowledge-based question answering:** Our enhanced knowledge graph can be used to answer complex questions by leveraging the combined knowledge of the graph and the LLM.
* **Recommendation systems:** By incorporating knowledge graph reasoning, our approach can improve the accuracy and explainability of recommendations.
* **Drug discovery and disease prediction:**  Our enhanced knowledge graph can be used to analyze relationships between drugs, diseases, and biological entities for potential drug discovery and disease prediction applications.
* **Natural language understanding:**  Our framework can be used to enhance natural language understanding tasks by providing a richer semantic understanding of text.

## Conclusion:

This project presents a novel and promising approach to knowledge graph completion by leveraging the capabilities of LLMs. We anticipate that our framework will contribute significantly to the advancement of knowledge representation and reasoning, leading to improvements in various applications across different domains.

## Project Plan:

**Phase 1 (1 month):**

* **Literature review and data collection:**  Study relevant papers on knowledge graph completion, LLMs, and knowledge graph embedding techniques.  Identify and select appropriate benchmark datasets.
* **Pre-training an LLM:** Choose a suitable LLM architecture and pre-train it on a large corpus of text and code.
* **Development of LLM-based knowledge graph embedding technique:**  Design and implement a method for generating contextualized entity and relation representations using the pre-trained LLM.

**Phase 2 (1 month):**

* **Implementation of knowledge graph completion model:** Train a knowledge graph completion model using the enriched embeddings generated in Phase 1. 
* **Evaluation and analysis:** Evaluate the performance of our proposed framework on benchmark datasets and compare it against state-of-the-art methods. Analyze the results to understand strengths and weaknesses of our approach.

**Phase 3 (1 month):**

* **Exploration of downstream applications:**  Investigate the potential of our enhanced knowledge graph for improving knowledge-based tasks like question answering and recommendation systems.
* **Project documentation and presentation:**  Prepare a comprehensive report summarizing our findings, including experimental results, analysis, and future directions.

## Topics to Study:

**Within 3 weeks, you should familiarize yourself with the following topics:**

* **Knowledge Graphs:** Basics of knowledge graphs, their structure, and applications.
* **Knowledge Graph Embedding:**  Different knowledge graph embedding techniques (TransE, RotatE, etc.) and their strengths and weaknesses.
* **Large Language Models:**  Understanding the architecture and capabilities of LLMs (e.g., BERT, GPT-3).
* **Pre-training and Fine-tuning LLMs:**  Methods for pre-training and fine-tuning LLMs for specific tasks.
* **Graph Neural Networks:**  Fundamentals of graph neural networks and their applications in knowledge graph completion.
* **Evaluation Metrics for Knowledge Graph Completion:**  Understanding relevant metrics for evaluating knowledge graph completion models (e.g., MRR, Hits@N).

This project provides a challenging yet rewarding experience that allows you to explore the cutting-edge intersection of LLMs and knowledge graphs. By successfully completing this project, you will gain valuable experience in research, development, and evaluation of innovative AI solutions. 


In [76]:
Markdown(llm.invoke(synopsis_format))

## Project Title:  Hyperbolic Knowledge Graph Attention Network for Multimodal Recommendation 

## Paper Title:  Hyperbolic Graph Attention Network 

## Abstract: 

This project aims to develop a novel multimodal recommendation system based on a Hyperbolic Knowledge Graph Attention Network (HKGAN).  HKGAN leverages the benefits of hyperbolic geometry to represent entities and relations in a knowledge graph, enhancing the ability to capture complex relationships and hierarchies within the data. By integrating multimodal information, such as text, images, and user preferences, HKGAN can provide more accurate and personalized recommendations. 

## Motivation:

* **Limitations of Euclidean Embeddings:** Traditional knowledge graph embedding methods using Euclidean space struggle to represent hierarchical structures and complex relationships between entities.
* **Hyperbolic Geometry for Enhanced Representation:** Hyperbolic geometry offers a more suitable space for modeling hierarchical structures and long-range dependencies, leading to more accurate embeddings.
* **Multimodal Information for Richer Context:** Incorporating multimodal information, such as images and text, provides a richer context for understanding user preferences and making more relevant recommendations.
* **Personalized and Explainable Recommendations:** HKGAN aims to provide personalized recommendations with explanations based on the learned relationships within the knowledge graph, improving user trust and satisfaction.

## Objectives: 

* **Developing a novel HKGAN architecture:** Design a specialized architecture that combines hyperbolic graph attention networks with multimodal information integration.
* **Hyperbolic Knowledge Graph Embedding:** Develop a method for embedding entities and relations in a hyperbolic space, capturing hierarchical structures and complex relationships.
* **Multimodal Information Fusion:** Integrate multimodal data sources, such as text, images, and user profiles, into the HKGAN architecture to enhance recommendation accuracy.
* **Personalized and Explainable Recommendations:**  Generate personalized recommendations with explainable reasoning based on the learned relationships in the knowledge graph.

## Architecture:

1. **Multimodal Feature Extraction:** Extract features from different modalities (text, images, user profiles) using appropriate pre-trained models.
2. **Hyperbolic Knowledge Graph Embedding:** Embed entities and relations in a hyperbolic space using a specialized embedding method.
3. **Hyperbolic Graph Attention Network:** Apply a hyperbolic graph attention network to learn relationships and preferences based on multimodal features and embedded entities.
4. **Recommendation Generation:** Generate personalized recommendations based on the learned relationships and preferences.
5. **Explanation Module:** Provide explanations for recommendations based on the learned relationships and multimodal information.

## Application Areas: 

* **E-commerce Recommendation:** Recommending products based on user preferences, product descriptions, and images.
* **Movie/Music Recommendation:** Recommending movies or music based on user preferences, genre information, and movie posters/album covers.
* **Social Media Recommendation:** Recommending relevant content and connections based on user interactions, social network graphs, and profile information.
* **Knowledge Graph Completion:** Predicting missing relationships in knowledge graphs based on multimodal information and learned embeddings.

## Conclusion:

This project aims to develop a novel HKGAN-based multimodal recommendation system that leverages the advantages of hyperbolic geometry and multimodal information integration. The project is expected to achieve high accuracy, personalization, and explainability in recommendation tasks, contributing significantly to the field of knowledge graph-based recommendation systems.

## Project Plan:

**Phase 1 (Weeks 1-2):**

* **Literature Review:** Conduct a comprehensive review of existing knowledge graph embedding methods, hyperbolic geometry, and multimodal information integration techniques.
* **Dataset Selection:** Choose a suitable multimodal dataset for recommendation tasks (e.g., Amazon product dataset, MovieLens dataset).
* **Feature Extraction:** Develop methods for extracting features from different modalities (text, images, user profiles).

**Phase 2 (Weeks 3-4):**

* **Hyperbolic Embedding:** Implement a hyperbolic embedding method for entities and relations in the chosen knowledge graph.
* **HKGAN Architecture:** Design and implement the HKGAN architecture incorporating hyperbolic attention mechanisms and multimodal information fusion.
* **Training and Evaluation:** Train the HKGAN model on the chosen dataset and evaluate its performance using relevant metrics (e.g., accuracy, precision, recall).

**Phase 3 (Weeks 5-6):**

* **Explainability Module:** Develop an explainability module to provide insights into the generated recommendations.
* **Application Development:** Integrate the HKGAN model into a user-friendly application for real-world recommendation scenarios.
* **Final Report and Presentation:** Prepare a comprehensive report and presentation summarizing the project findings and contributions.

## Topics to Study:

* **Knowledge Representation:** Concepts of knowledge graphs, entity-relationship modeling, and knowledge graph embedding methods.
* **Graph Neural Networks:** Basic concepts, architectures, and applications of GNNs.
* **Hyperbolic Geometry:** Introduction to hyperbolic geometry, its properties, and its applications in machine learning.
* **Multimodal Information Fusion:** Techniques for integrating and processing data from multiple modalities.
* **Recommendation Systems:** Principles of recommendation systems, evaluation metrics, and common techniques.

**Note:** This project plan provides a general framework. The specific details and tasks may vary depending on the chosen dataset, chosen embedding method, and the specific focus of the project.
